In [11]:

import os
import json
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage
from Utils.Prompt import planner_no_route_agent

OPENAI_API_KEY = os.getenv('OPEN_AI_API')

class SolePlanning:
    def __init__(self,
                 planner_llm) -> None:
        
        self.planner_llm = planner_llm

        if self.planner_llm == 'gpt-4o-2024-11-20':
            self.llm = ChatOpenAI(temperature=0,
                        model_name='gpt-4o-2024-11-20',
                        openai_api_key=OPENAI_API_KEY)
            
    def createPlan(self, prompt):
        if self.planner_llm == 'gpt-4o-2024-11-20':
            request = self.llm.invoke([HumanMessage(prompt)]).content
            return request


if __name__ == '__main__':
    modelList = ['gpt4o','gemini']
    model_map = {'gpt4o': 'gpt-4o-2024-11-20'}
    taskList = ['allDataNoRoute', 'allDataRoutOP', 'filteredDataRouteOP']

    #just choose the model and the task
    #choose model
    model = modelList[0]
    planner_llm = model_map[model]
    print("the model you choose is: ", model, " The planner LLM is: ", planner_llm)
    # choose task
    task = taskList[0]
    print("The task you choose is: ", task)
    #number of plan
    numPlan = 20
    print("You will run: " + str(numPlan) + " plans")
    

    

    #load data (all/filtered)
    if 'all' in task:
        with open(f'Dataset/{model}/all_data.json', 'r') as file:
            given_information = json.load(file)
    else:
        with open(f'Dataset/{model}/filtered_data.json', 'r') as file:
            given_information = json.load(file)
    
    #iterate through all human querys
    plans = []
    for i in range(numPlan):
        #load human query, allData only 100 plans, filteredData 500 plans.
        with open ('Prompts/humanQuerys.jsonl', 'r') as file:
            humanquerys = [json.loads(line.strip()) for line in file]
        query = humanquerys[i]['query']
        #print(query)
        
        #load prompt agent, and make the prompt
        if task == 'allDataNoRoute':
            prompt_agent = planner_no_route_agent
            prompt = prompt_agent.format(given_information = given_information, query = query)
            #print(prompt)
        

        #actual inference
        solePlanning = SolePlanning(planner_llm=planner_llm)
        plan = {"index": i+1, "plan": solePlanning.createPlan(prompt)}
        plans.append(plan)

    with open (f'Output/{model}/plans/{task}.jsonl', 'w') as file:
        for plan in plans:
            file.write(json.dumps(plan) + '\n')

the model you choose is:  gpt4o  The planner LLM is:  gpt-4o-2024-11-20
The task you choose is:  allDataNoRoute


In [4]:
query

'I am planning a 3-day trip with an expensive budget. I would like to explore some nature-oriented attractions. Please recommend restaurants that serve US cuisine and offer good value. For accommodation, I am looking for a hotel with good service.'